<a href="https://colab.research.google.com/github/bryanjiang1204/yes/blob/main/Video_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fvcore
!pip install av
!pip install pytorchvideo

In [60]:
import torch

model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)

device = "cpu"
model = model.eval()
model = model.to(device)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [61]:
import json
import urllib
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
frames_per_second = 30


transform_params = {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 13,
        "sampling_rate": 6,
    }

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params["num_frames"]),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=transform_params["side_size"]),
            CenterCropVideo(
                crop_size=(transform_params["crop_size"], transform_params["crop_size"])
            )
        ]
    ),
)

clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second

In [62]:
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try: urllib.URLopener().retrieve(json_url, json_filename)
except: urllib.request.urlretrieve(json_url, json_filename)

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [63]:
url_link = "https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4"
video_path = 'archery.mp4'
try: urllib.URLopener().retrieve(url_link, video_path)
except: urllib.request.urlretrieve(url_link, video_path)

In [64]:
start_sec = 1
clip_duration = 3
end_sec = start_sec + clip_duration

video = EncodedVideo.from_path(video_path)

video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
video_data = transform(video_data)

inputs = video_data["video"]
inputs = inputs.to(device)

In [70]:
preds = model(inputs[None,...])

post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=3).indices[0]

pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
print("Top predicted label: %s" % ", ".join(pred_class_names))

Top predicted label: archery, air drumming, abseiling


In [71]:
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

train_nodes, eval_nodes = get_graph_node_names(model)

class Video_Embeddings(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Get a resnet50 backbone
        model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True).eval()

        # Extract 4 main layers (note: MaskRCNN needs this particular name
        # mapping for return nodes)
        self.body = create_feature_extractor(
            model, return_nodes={'blocks.5.proj': 'embeddings',})
        # Dry run to get number of channels for FPN

    def forward(self, x):
        x = self.body(x)
        return x

embedder = Video_Embeddings()
intermediate_layer = embedder(inputs[None,...])
intermediate_layer['embeddings'].shape


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


torch.Size([1, 1, 2, 2, 400])